In [2]:
import pandas as pd
import re
from tqdm import tqdm
import math
import numpy as np
import json
from ast import literal_eval

In [3]:
df = pd.read_excel("Song_Offices_RegEx.xlsx", sheet_name="Central")
df.dropna(inplace=True)

In [4]:
prefix = "[權發遣|添差|前|權|攝|充|贈|累贈|檢校|試]?"

In [5]:
df['len'] = df['c_office_chn'].apply(len)

In [6]:
df = df.sort_values("len", ascending=False, ignore_index=True)

In [7]:
# All QSW records.
df_qsw_raw=pd.read_excel('../data_raw/quan_song_wen_muzhi.xlsx', sheet_name='墓誌銘墓表壙誌行狀神道碑塔銘墓碑')[['content_id', 'content', 'subject', 'author']].set_index('content_id')
df_qsw_raw['content']=[str(s) for s in df_qsw_raw['content']]
df_qsw_raw.dropna(inplace=True)
df_qsw_raw = df_qsw_raw[df_qsw_raw['content'] != "nan"]

/usr/remote/anaconda-3.7-deeplearn/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:300: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [8]:
# Function for seperating sentences.
def sep_mark_sent(string):
    string=string.replace('，', '/ws1ep/').replace('；', '/ws2ep/').replace('、', '/ws3ep/')
    string=string.replace('！', '/wend/').replace('。', '/wend/').replace('？', '/wend/').replace('！', '/wend/')
    string=string.replace('：', '/wm/')
    return [s for s in string.split("/wend/") if s!='']

In [9]:
# Seperate sentences and retain sentences with kinship words.
all_sent_count=0
kin_sent_count=0
df_qsw_refined=pd.DataFrame()
for index in tqdm(df_qsw_raw.index):
    content=df_qsw_raw.loc[index]['content']
    sent_list=sep_mark_sent(content) # Seperate sentences.
    all_sent_count+=len(sent_list)
    subject=df_qsw_raw.loc[index]['subject']
    author=df_qsw_raw.loc[index]['author']
    content_id=index
    # Retain sentences with kinship information.
    df_qsw_refined=pd.concat([pd.DataFrame(data=[[content_id, subject, author, sent, content] for sent in sent_list],
                                          columns=['content_id', 'subject', 'author', 'sent', 'content']
                                          ), df_qsw_refined], axis=0, ignore_index=True
                            )
# Run on entire corpus, 21.2%.

100%|██████████| 4701/4701 [01:49<00:00, 42.89it/s] 


In [10]:
for i in tqdm(df_qsw_refined.index):
    row = df_qsw_refined.loc[i]
    title = []
    for noc in df['c_office_chn'].values:
        if noc in row['sent']:
            re_rule = prefix + noc
            l = re.findall(re_rule, row['sent'])
            row['sent'] = row['sent'].replace(l[0], "/noc/")
            title.append(l[0])
    if title != []:
        df_qsw_refined.loc[i, "no_noc"] = str(title)

  0%|          | 0/258714 [00:00<?, ?it/s]/home/jc092/.local/lib/python3.7/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
100%|██████████| 258714/258714 [1:49:11<00:00, 39.49it/s]  


In [13]:
df_qsw_refined.dropna(inplace = True)

In [15]:
df_qsw_refined.to_excel("central_title_list.xlsx")